In [ ]:
from sklearn.datasets import load_iris
import pandas as pd

# Load the Iris dataset
iris = load_iris()

# Convert to pandas DataFrame
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Separate features (X) and target (y)
X = df[iris.feature_names]
y = df['target']

# Display the first few rows of X and y
print("Features (X):")
display(X.head())

print("\nTarget (y):")
display(y.head())

 


Features (X):


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2



Target (y):


0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instantiate a LogisticRegression model for the target model
target_model = LogisticRegression(max_iter=1000)

# Train the target model on the full dataset
target_model.fit(X, y)

print("Target model trained successfully.")
 


Target model trained successfully.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the original dataset into shadow train/test sets
X_shadow_train, X_shadow_test, y_shadow_train, y_shadow_test = train_test_split(
    X, y, test_size=0.5, random_state=42, stratify=y)

# Scale features for the shadow model
scaler_shadow = StandardScaler()
X_shadow_train = pd.DataFrame(
    scaler_shadow.fit_transform(X_shadow_train),
    columns=X.columns)
X_shadow_test = pd.DataFrame(
    scaler_shadow.transform(X_shadow_test),
    columns=X.columns)

# Train the shadow model
shadow_model = LogisticRegression(max_iter=1000)
shadow_model.fit(X_shadow_train, y_shadow_train)

print("Shadow model trained.")

Shadow model trained.


In [ ]:
# 1. Generate predictions and probabilities from the shadow model
shadow_train_pred_proba = shadow_model.predict_proba(X_shadow_train)
shadow_test_pred_proba = shadow_model.predict_proba(X_shadow_test)

# 2. Create labels for shadow train & test data
shadow_train_labels = pd.Series([1] * len(X_shadow_train))   # 1 = in-members
shadow_test_labels = pd.Series([0] * len(X_shadow_test))     # 0 = out-members

# 3. Convert prediction probs into DataFrames
shadow_train_pred_proba_df = pd.DataFrame(shadow_train_pred_proba)
shadow_test_pred_proba_df = pd.DataFrame(shadow_test_pred_proba)

# Add labels as new column
shadow_train_pred_proba_df['is_member'] = shadow_train_labels.values
shadow_test_pred_proba_df['is_member'] = shadow_test_labels.values

# 4. Concatenate train & test for attack model training
attack_train_data = pd.concat(
    [shadow_train_pred_proba_df, shadow_test_pred_proba_df],
    ignore_index=True
)

# 5. Separate features and target
X_attack_train = attack_train_data.drop('is_member', axis=1)
y_attack_train = attack_train_data['is_member']

print("Prepared training data for the attack model.")
display(X_attack_train.head())
display(X_attack_train.tail())
display(y_attack_train.head())
display(y_attack_train.tail())

 


Prepared training data for the attack model.


,0,1,2
0,0.984571,0.015424,0.000006
1,0.057128,0.925947,0.016925
2,0.000125,0.046193,0.953682
3,0.020664,0.851088,0.128247
4,0.932282,0.067699,0.000018


,0,1,2
145,0.025750,0.891341,8.290922e-02
146,0.993220,0.006779,9.885163e-07
147,0.000088,0.075448,9.244637e-01
148,0.000474,0.037352,9.621744e-01
149,0.010591,0.419520,5.698886e-01


0    1
1    1
2    1
3    1
4    1
Name: is_member, dtype: int64

145    0
146    0
147    0
148    0
149    0
Name: is_member, dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression

# 4. Instantiate a binary classifier (Logistic Regression) as the attack model
attack_model = LogisticRegression()

# 5. Train the attack model
attack_model.fit(X_attack_train, y_attack_train)

print("Attack model trained successfully.")
 


Attack model trained successfully.


In [ ]:
from sklearn.metrics import accuracy_score

# 1. Generate prediction probabilities for the full dataset X
target_pred_proba = target_model.predict_proba(X)

# 2. Create a DataFrame from prediction probabilities
target_pred_proba_df = pd.DataFrame(target_pred_proba)

# 3. Use the attack model to predict membership (is_member)
predicted_membership = attack_model.predict(target_pred_proba_df)

# 4. True membership labels:
# All points used to train target_model → membership = 1
actual_membership = [1] * len(X)

# 5. Calculate accuracy of membership inference attack
accuracy = accuracy_score(actual_membership, predicted_membership)
print(f"Attack model accuracy: {accuracy}")

 


Attack model accuracy: 0.35333333333333333
